In [1]:
!pip install -q oauth2client==1.5.2
import sys
sys.path.append("./utils/student_function/")
from mooclib import *
import inspect, urllib
html, auth_code, userinfo = google_authenticate(PORT_NUMBER=8080)
html

CLICK ON THIS LINK TO AUTHENTICATE WITH YOUR GMAIL ACCOUNT
https://accounts.google.com/o/oauth2/auth?scope=profile+email&redirect_uri=http%3A%2F%2Flocalhost%3A8080&response_type=code&client_id=486275613361-7nk9ktukfcfjevfoe3tamjp0pmbjmprs.apps.googleusercontent.com&access_type=offline
waiting for authentication ...
authentication succeeded


,daniel.riveraa@udea.edu.coDANIEL FELIPE RIVERA ARROYAVEgoogle id: 107960807606835866040authorization code: 4/ZgAmtLUcBP9bekY9NpEQr3A6g24JxkEv3BbpH4-Q1HgjrZ6YAC5vnn4u3dOmLuwGBm3aCinQ-81jtbIclNoRh-E


En estos ejercicios usaremos las clases `Node` y `L`. Ejecuta la siguiente celda para cargar sus definiciones

In [5]:
%%writefile ./utils/student_function/PS05.py
class Node:
    def __init__(self, value, next=None):
        assert next is None or isinstance(next,Node), "next must be Node, not %s"%(type(next))
        self.value = value
        self.next  = next
        
    def __repr__(self):
        return str(self.value)
    
class L(object):
    def __init__ (self, first_node=None):
        assert first_node is None or isinstance(first_node,Node), "first must be Node, not %s"%(type(first_node))
        self.first_node = first_node
        
    def __getitem__(self, idx):
        k = self.first_node
        for i in range(idx):
            assert k.next is not None, "index %s out of range"%(str(idx))
            k = k.next  
        return k.value
    
    def __len__(self):
        k = self.first_node
        if k is None:
            return 0
        i=1
        while k.next is not None:
            i+=1
            k = k.next
        return i
            
    def __repr__ (self):
        if self.first_node is None:
            return "[]"
        
        s = "[ %s"%self.first_node
        k=self.first_node
        while (k.next is not None):
            s += ", %s"%k.next
            k = k.next
    
        return s+" ]"

Writing ./utils/student_function/PS05.py


## Ejercicio 1

Implementa el método `__getitem__` para que funcione con índices positivos y negativos. Por ejemplo:

    > W = L1(Node(10, Node(20, Node(30))))
    > print W
    [ 10, 20, 30 ]

    > print W[0], W[1], W[2]
    > print W[-1], W[-2], W[-3]
    10 20 30
    30 20 10

Ten en cuenta los casos en los que el índice excede el tamaño de la lista, bien positivo bien negativo. **En estos casos has de generar un assertion error**. 

En el ejemplo anterior, `W[3]` y `W[-4]` deben de generar un `AssertionError`

In [39]:
%%writefile ./utils/student_function/PS05_01.py
from PS05 import *
class L1(L):
    def __getitem__(self, idx):
        k = self.first_node
        if idx >=0:
            for i in range(idx):
                assert k.next is not None, "index %s out of range"%(str(idx))
                k = k.next
        else:
            lenNegative=(len(self)*-1)
            assert idx>=lenNegative, "te pasas we"
            for i in range(len(self)+idx):
                assert k.next is not None, "index %s out of range"%(str(idx))
                k = k.next  
        return k.value


Overwriting ./utils/student_function/PS05_01.py



comprueba manualmente tu código

In [40]:
from PS05 import *
import PS05_01 as st
reload(st)

W = st.L1(Node(10, Node(20, Node(30))))
print W

[ 10, 20, 30 ]


In [41]:
print W[0], W[1], W[2]
print W[-1], W[-2], W[-3]

10 20 30
30 20 10


In [42]:
print W[3]

AssertionError: index 3 out of range

In [43]:
print W[-4]

AssertionError: te pasas we

#### comprueba tu código con el corrector automático

In [44]:
!./run CHECK_SOLUTION PS05_01


** CORRECTO **
evaluation result 5.0


#### registra tu solución en línea

In [45]:
!./run SUBMIT_SOLUTION PS05_01

connecting ... registering submission for daniel.riveraa@udea.edu.co ...
no config found for section PS05, allowing all submissions

** CORRECTO **
your submissions sheet is https://docs.google.com/spreadsheets/d/1QjA5eGBEkEfHxuQDJaqn7YQXH-ECIalATeOgF8dw-nU
----
evaluation result 5.0 , submission registered


## Ejercicio 2

Implementa una función `__setitem__` que también soporte índices negativos. Por ejemplo:

    W = L2(Node(10, Node(20, Node(30))))
    print W
    [ 10, 20, 30 ]

    W[1]=25
    print W
    [ 10, 25, 30 ]

    W[-1]=35
    print W
    [ 10, 25, 35 ]


In [48]:
%%writefile ./utils/student_function/PS05_02.py
from PS05 import *
class L2(L):
    def __setitem__(self, idx, value):
        k = self.first_node
        if idx>=0:
            for i in range(idx):
                assert k.next is not None, "index %s out of range"%(str(idx))
                k = k.next
            k.value=value        
        else:
            for j in range(len(self)+idx):
                assert k.next is not None, "index %s out of range"%(str(idx))
                k = k.next
            k.value=value    
        return k.value
        

Overwriting ./utils/student_function/PS05_02.py


In [49]:
from PS05 import *
import PS05_02 as st
reload(st)

W = st.L2(Node(10, Node(20, Node(30))))
print W

[ 10, 20, 30 ]


In [50]:
W[1]=25
print W

[ 10, 25, 30 ]


In [51]:
W[-1]=35
print W

[ 10, 25, 35 ]


#### comprueba tu código con el corrector automático

In [52]:
!./run CHECK_SOLUTION PS05_02


** CORRECTO **
evaluation result 5.0


#### registra tu solución en línea

In [53]:
!./run SUBMIT_SOLUTION PS05_02

connecting ... registering submission for daniel.riveraa@udea.edu.co ...
no config found for section PS05, allowing all submissions

** CORRECTO **
your submissions sheet is https://docs.google.com/spreadsheets/d/1QjA5eGBEkEfHxuQDJaqn7YQXH-ECIalATeOgF8dw-nU
----
evaluation result 5.0 , submission registered


## Ejercicio 3

Implementa la función `clone_node` que duplica. Si el nodo tiene un `next` también lo duplica

Sugerencia: implementa el siguiente pseudocódigo **recursivo**

    function clone_node(node)
     
        si node es NULL
            devuelve NULL
            
        devuelve nuevo_nodo(node.value, clone_node(node.next))

In [79]:
%%writefile ./utils/student_function/PS05_03.py
from PS05 import *

def clone_node(node):
    if(node == None):
        return None    
    return Node(node.value, clone_node(node.next))

Overwriting ./utils/student_function/PS05_03.py


comprueba manualmente tu código. Los valores han de ser iguales pero los `id` no, señalando que son verdaderamente objetos distintos

In [88]:
from PS05 import *
import PS05_03 as st
reload(st)

n = Node(10, Node(20, Node(30)))
print n, id(n)
print n.next, id(n.next)
print n.next.next, id(n.next.next)
print n.next.next.next, id(n.next.next.next)

10


In [81]:
k = st.clone_node(n)
print k, id(k)
print k.next, id(k.next)
print k.next.next, id(k.next.next)
print k.next.next.next, id(k.next.next.next)

10 140200094937384
20 140200094938176
30 140200094937168
None 140200564985280


#### comprueba tu código con el corrector automático

In [82]:
!./run CHECK_SOLUTION PS05_03


** CORRECTO **
evaluation result 5.0


#### registra tu solución en línea

In [85]:
!./run SUBMIT_SOLUTION PS05_03

connecting ... registering submission for daniel.riveraa@udea.edu.co ...
no config found for section PS05, allowing all submissions

** CORRECTO **
your submissions sheet is https://docs.google.com/spreadsheets/d/1QjA5eGBEkEfHxuQDJaqn7YQXH-ECIalATeOgF8dw-nU
----
evaluation result 5.0 , submission registered


## Ejercicio 4

Implementa `clone` que duplica una lista. Pon atención, ya que si hay sublistas, éstas han de ser duplicadas también. Entiende la implementación de la siguiente función, que muestra recursivamente los `id` de cada nodo de una lista **según tu propia implementación del ejercicio 2**

In [20]:
from PS05 import *
import PS05_02 as st2
def show_ids(M, level=0):
    k = M.first_node
    while k is not None:
        print " "*2*level, id(k)
        if isinstance(k.value, M.__class__):
            show_ids(k.value, level+1)
        k = k.next
        

In [21]:
import PS05_02 as st2
from PS05 import *

W=st2.L2(Node(10, Node(st2.L2(Node(14, Node(15, Node(st2.L2(Node(16, Node(17))))))), Node(20, Node(30)))))
show_ids(W)

 139669160335192
 139669160335120
   139669160334904
   139669160334832
   139669160334760
     139669160334544
     139669160334616
 139669160335048
 139669160334976


Desarrolla tu solución de la siguiente manera:

1. Primero copia los nodos de la lista actual (`self`)
1. Crea una lista nueva con los nodos copiados
1. Recorre los nodos de la lista nueva verificando el campo `value`
1. Si este campo es también una lista (usa `isinstance` como en la función `show_ids`) entonces llama a `clone` de esa lista y sustituye el valor.

In [112]:
%%writefile ./utils/student_function/PS05_04.py
from PS05 import *
class L4(L):
    def clone(self):
        def clone_node(node):
            if node == None:
                return None    
            return Node(node.value, clone_node(node.next))
                    
        r=L4(clone_node(self.first_node))
        current=r.first_node
        for i in range(len(r)):
            if isinstance(current.value,st.L4):
                current.value=st.L4(clone_node(current.value.first_node))
            current=current.next
        return r

Overwriting ./utils/student_function/PS05_04.py


comprueba tu código, ambas listas han de tener el mismo contenido, pero distintos `id`

In [115]:
import PS05_04 as st
reload(st)
from PS05 import *

W=st.L4(Node(10, Node(st.L4(Node(14, Node(15, Node(st.L4(Node(16, Node(17))))))), Node(20, Node(30)))))
W

[ 10, [ 14, 15, [ 16, 17 ] ], 20, 30 ]

In [116]:
K = W.clone()
print K

[ 10, [ 14, 15, [ 16, 17 ] ], 20, 30 ]


In [117]:
show_ids(W)

 139669159418408
 139669159418336
   139669159418120
   139669159416752
   139669159417904
     139669159417976
     139669160820104
 139669159418264
 139669159418192


In [118]:
show_ids(K)

 139669159418984
 139669159418912
   139669159419200
   139669159419128
   139669159419056
     139669159417976
     139669160820104
 139669159418624
 139669159418552


#### comprueba tu código con el corrector automático

In [119]:
!./run CHECK_SOLUTION PS05_04


** CORRECTO **
evaluation result 5.0


#### registra tu solución en línea

In [122]:
!./run SUBMIT_SOLUTION PS05_04

connecting ... registering submission for daniel.riveraa@udea.edu.co ...
no config found for section PS05, allowing all submissions

** CORRECTO **
your submissions sheet is https://docs.google.com/spreadsheets/d/1QjA5eGBEkEfHxuQDJaqn7YQXH-ECIalATeOgF8dw-nU
----
evaluation result 5.0 , submission registered


## Ejercicio 5

Implementa el método ´__add__´ para que concatene una lista con otra creando una nueva lista. Usa los métodos desarrollados en los ejercicios anteriores. 

**Sugerencia de implementación**: copia primero las dos listas originales y luego concaténalas haciendo apuntar el `next` del último elemento de la copia de la primera lista al primer elemento de la copia de la segunda lista.

In [129]:
%%writefile ./utils/student_function/PS05_05.py
from PS05 import *
class L5(L):
    
    def clone(self):
        def clone_node(node):
            if node==None:
                return None
            return Node(node.value, clone_node(node.next))         
               
        r=L5(clone_node(self.first_node))
        x=r.first_node
        for i in range(len(r)):
            if isinstance(x.value,L5):
                x.value=L4(clone_node(x.value.first_node))
            x=x.next            
        return r
    
    def __add__(self, M):
        a=self.clone()
        b=M.clone()
        i=a.first_node
        while i.next is not None:
            i=i.next
        i.next=b.first_node        
        return a  
        

Overwriting ./utils/student_function/PS05_05.py


In [131]:
import PS05_05 as st
reload(st)
from PS05 import * 
W = st.L5(Node(10, Node(20, Node(30))))
Z = st.L5(Node(3, Node(2)))
print W
print Z
print W + Z + st.L5() 

[ 10, 20, 30 ]
[ 3, 2 ]
[ 10, 20, 30, 3, 2 ]


esto ahora debería de funcionar (recuerda que en las Notas correspondientes existía un problema)

In [132]:
X = W + Z + W
print X

[ 10, 20, 30, 3, 2, 10, 20, 30 ]


#### comprueba tu código con el corrector automático

In [133]:
!./run CHECK_SOLUTION PS05_05


** CORRECTO **
evaluation result 5.0


#### registra tu solución en línea

In [134]:
!./run SUBMIT_SOLUTION PS05_03

connecting ... registering submission for daniel.riveraa@udea.edu.co ...
no config found for section PS05, allowing all submissions

** CORRECTO **
your submissions sheet is https://docs.google.com/spreadsheets/d/1QjA5eGBEkEfHxuQDJaqn7YQXH-ECIalATeOgF8dw-nU
----
evaluation result 5.0 , submission registered


## Ejercicio 6

Implementa la función de igualdad.

**Sugerencia de implementación**: haz una función que recorra simultáneamente los nodos de ambas listas (`self` y `other`) y devuelva `False` en cuanto encuentre que el campo `value` de los dos nodos sea distinto (`!=`). Pon especial atención a que si `value` es a su vez una lista, el método será recursivo.

In [2]:
%%writefile ./utils/student_function/PS05_06.py
from PS05 import *
class L6(L):
    
    def __eq__(self, other):
            
        a=self.first_node;b=other.first_node
        if len(self) !=len(other):return False
        for i in range(len(self)):
            if a.value!=b.value:return False
            a=a.next;b=b.next
        return True 
    
    def __ne__(self, other):
        return not self==other       

Writing ./utils/student_function/PS05_06.py


In [3]:
import PS05_06 as st
from PS05 import *
reload(st)
W=st.L6(Node(10, Node(st.L6(Node(14, Node(15, Node(st.L6(Node(16, Node(17))))))), Node(20, Node(30)))))
Z=st.L6(Node(10, Node(st.L6(Node(14, Node(15, Node(st.L6(Node(16, Node(17))))))), Node(20, Node(30)))))
Y=st.L6(Node(10, Node(st.L6(Node(14, Node(15))), Node(20, Node(30)))))
print W
print Z
print Y

[ 10, [ 14, 15, [ 16, 17 ] ], 20, 30 ]
[ 10, [ 14, 15, [ 16, 17 ] ], 20, 30 ]
[ 10, [ 14, 15 ], 20, 30 ]


In [4]:
print W==W
print Z==Z
print Y==Y
print W==Z
print W==Y
print Y==Z

True
True
True
True
False
False


#### comprueba tu código con el corrector automático

In [5]:
!./run CHECK_SOLUTION PS05_06


** CORRECTO **
evaluation result 5.0


#### registra tu solución en línea

In [6]:
!./run SUBMIT_SOLUTION PS05_06

connecting ... registering submission for daniel.riveraa@udea.edu.co ...
no config found for section PS05, allowing all submissions

** CORRECTO **
your submissions sheet is https://docs.google.com/spreadsheets/d/1QjA5eGBEkEfHxuQDJaqn7YQXH-ECIalATeOgF8dw-nU
----
evaluation result 5.0 , submission registered


## Ejercicio 7

Realiza una función que evalúe un `VarTerm` dado un diccionario `vals`. Asume que este diccionario contendrá un valor para la variable del término. Por ejemplo

    > vals = {"x": 1, "y":2, "z": 3}
    > vt = VarTerm(9,"y", 2)
    > print vt, "=", evaluate_vterm(vt, vals)
    9 y^2 = 36
    
    > vt = VarTerm(2,"z", 3)
    > print vt, "=", evaluate_vterm(vt, vals)
    2 z^3 = 54


In [7]:
%%writefile ./utils/student_function/PS05_07.py
from IPython.display import Math

class VarTerm():
    def __init__(self, coef, var, exp=1):
        assert (isinstance(coef, float) or isinstance(coef,int)) and \
                isinstance(exp, int) and isinstance(var, str) and len(var)==1
        self.coef = coef
        self.var  = var
        self.exp  = exp
        
    def get_math_representation(self):
        s = ("%s"%str(self.coef) if self.coef!=1 else "") +\
            (" %s"%self.var if self.exp==1 else (" %s^%d"%(self.var, self.exp) if self.exp!=0 else "")) 
        return s
    
    def __repr__(self):
        return self.get_math_representation() 
        
    def evaluate(self, vals):
        var=vals[self.var]
        return self.coef*(var**self.exp)

Writing ./utils/student_function/PS05_07.py


In [8]:
import PS05_07 as st
reload(st)

vals = {"x": 1, "y":2, "z": 3}
vt = st.VarTerm(9,"y", 2)
print vt, "=", vt.evaluate(vals)
vt = st.VarTerm(2,"z", 3)
print vt, "=", vt.evaluate(vals)

9 y^2 = 36
2 z^3 = 54


#### comprueba tu código con el corrector automático

In [9]:
!./run CHECK_SOLUTION PS05_07


** CORRECTO **
evaluation result 5.0


#### registra tu solución en línea

In [10]:
!./run SUBMIT_SOLUTION PS05_07

connecting ... registering submission for daniel.riveraa@udea.edu.co ...
no config found for section PS05, allowing all submissions

** CORRECTO **
your submissions sheet is https://docs.google.com/spreadsheets/d/1QjA5eGBEkEfHxuQDJaqn7YQXH-ECIalATeOgF8dw-nU
----
evaluation result 5.0 , submission registered


## Ejercicios de refuerzo (no obligatorios)

- crea una función que evalúe un `VarPol`, según definido en las notas.
- analiza la complejidad computacional de tus implementaciones
- ¿Podrías enriqueces `Node` para mejorar la complejidad computacional?